In [1]:
# %matplotlib
from nanorsm_v2 import *
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%5|1740585423.514|CONFWARN|rdkafka#producer-1| [thrd:app]: No `bootstrap.servers` configured: client will not be able to connect to Kafka cluster


# Load XRF data for stack registration

In [3]:
# load elemental image data, align it with pystackreg and generate a tranform matrix
#put xrf data with a filena

det_params = {'merlin1':55, "merlin2":55, "eiger":75}

logfile = '//data//users//current_user/Find_ideal_theta_scan_startID330770.csv'

xrf_folder = '//data//users//current_user//XRF//'
elem = 'Ta_L'
diff_detector = 'merlin1'

df = pd.read_csv(logfile)
sid_list = df['scan_id'].to_numpy(dtype = 'int')
angles = df['angle'].to_numpy()
#sid_list = get_sid_list(['218921-219056'],3)


num = np.size(sid_list)
file_list = []
for sid in sid_list:
    
    tmp = os.path.join(xrf_folder, f'output_tiff_scan2D_{int(sid)}/detsum_{elem}_norm.tiff')
    print(tmp)
    file_list.append(tmp)
im_stack = load_ims(file_list)

num_frame,im_row,im_col = np.shape(im_stack)

im_stack_aligned, trans_matrix = align_im_stack(im_stack) # use pystackreg

#im_stack = np.delete(im_stack,0,axis=0)
#trans_matrix = np.delete(trans_matrix,0,axis=0)

im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly


dir_ = os.path.abspath(os.path.dirname(logfile))
folder_name = os.path.basename(logfile).split('.')[0]
save_folder =  os.path.join(dir_,folder_name+"_diff_data")
data_path = save_folder

os.makedirs(save_folder, exist_ok = True)
                                
xrf_savename = os.path.join(save_folder,f"aligned_xrf_stack_{elem}.tiff")

tf.imwrite(xrf_savename,im_stack_test.astype(np.float32),imagej=True)

//data//users//current_user//XRF//output_tiff_scan2D_330770/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330772/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330774/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330776/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330778/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330780/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330782/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330784/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330786/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330788/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330790/detsum_Ta_L_norm.tiff
//data//users//current_user//XRF//output_tiff_scan2D_330792/detsum_Ta_L_norm.tiff
//data//users//c

Progress:   0%|          | 0/31 [00:00<?, ?it/s]

Progress:   0%|          | 0/31 [00:00<?, ?it/s]

# Apply stack alignemnt to other elemenets (Optional)

In [6]:
# if there are more elements to be aligned and stacked
elem_list = ['Ni_K']
#sid_list = get_sid_list(['219792-219888'],3)

num = np.size(sid_list)
i = 1
for elem in elem_list:
    file_list = []
    for sid in sid_list:
        tmp = os.path.join(xrf_folder, f'output_tiff_scan2D_{int(sid)}/detsum_{elem}_norm.tiff')
        
        file_list.append(tmp)
    im_stack = load_ims(file_list)
    
    im_stack_test = interp_sub_pix(im_stack,trans_matrix)
    imp = np.sum(im_stack_test,0)
    imp.shape
    if i == 1:
        stack = imp
        sz = np.shape(imp)
        stack = np.reshape(stack, (1,sz[0],sz[1]))
    else:
        stack = np.concatenate((stack,imp[np.newaxis,:,:]),0)
    i = i+1
    xrf_savename = os.path.join(save_folder,f"aligned_xrf_stack_{elem}.tiff")
    tf.imwrite(xrf_savename,im_stack_test.astype(np.float32),imagej=True)

Progress:   0%|          | 0/31 [00:00<?, ?it/s]

Progress:   0%|          | 0/31 [00:00<?, ?it/s]

# Load a total sum of diffraction images to decide the ROI and any mask for diff images (Optional if ROI is decided before)

In [ ]:
#sid_list = get_sid_list(['219422'],1)
tot = sum_all_h5_data_db(sid_list,diff_detector) 
plt.figure()
plt.imshow(np.log10(tot))
plt.savefig(os.path.join(save_folder,f"total_sum_{diff_detector}_image.png"))
plt.show()

Progress:   0%|          | 0/31 [00:00<?, ?it/s]

/tmp/ipykernel_2890924/1617494443.py:4: RuntimeWarning: divide by zero encountered in log10
  plt.imshow(np.log10(tot))


# Apply a mask (Optional)

In [8]:
# mask = np.asarray(tifffile.imread('//data//users//2023Q2//Murray_2023Q2//nanodiffraction//mask.tif'))
# plt.figure()
# plt.imshow(np.log10(mask),clim=[0,5])
plt.imshow(np.log10(tot*mask+0.1),clim=[0.1,3.5])
#tifffile.imwrite('/data/users/2023Q2/Murray_2023Q2/Substrate_35_perc_sum_all.tif',tot.astype(np.float32))

/opt/conda_envs/nsls2-analysis-2021-1.2/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


# sum all diffraction patterns from local files to determine roi and mask (Optional)

In [ ]:

# data_path = '//data//home//home//hyan//export//scan_180000//'
#prefix = 'scan_'
#postfix = '_merlin1.h5'

#data_file_list = create_file_list(data_path, prefix, postfix, sid_list)
#tot = sum_all_h5_data(data_file_list)
#plt.imshow(np.log10(tot+0.1),clim=[1,5])

In [ ]:
# determine from tot image
#mask = np.ones(tot.shape)

#plt.imshow(np.log10(tot*mask+0.1))
#roi = [185,105,160,130]

# Check the ROI image

In [14]:
roi = [80,80,100,100]
plt.imshow(np.log10(tot[roi[0]:roi[0]+roi[2],roi[1]:roi[1]+roi[3]]))
#mask = np.asarray(tifffile.imread('//data//users//2023Q2//Murray_2023Q2//nanodiffraction//mask.tif'))
plt.savefig(os.path.join(save_folder,f"ROI_total_sum_{diff_detector}_image.png"))
plt.show()

NameError: name 'tot' is not defined

# Load, normalize, align, and assemble diffraction data through databroker
# diff_data is the 5D dataset aligned; make sure your machine has enough memory

In [15]:

scan_row,scan_col = get_diff_data_shape(sid_list)
diff_data = load_h5_data_db(sid_list,det='merlin1', mon='sclr1_ch4', roi=roi)

sz = diff_data.shape
diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)

Progress:   0%|          | 0/21 [00:00<?, ?it/s]

Total scan points: 40000; raw image row: 247; raw image col: 255
Total scan points: 40000; data image row: 100; data image col: 100


Progress:   0%|          | 0/21 [00:00<?, ?it/s]

# check if images look normal

In [16]:
plt.figure()
plt.imshow(diff_data[5,20,20,:,:])
plt.show()

In [ ]:
# Load, normalize, align, and assemble diffraction data 
# diff_data is the 5D dataset aligned; make sure your machine has enough memory
'''
data_path = '//data//home//home//hyan//export//scan_180000//'
prefix = 'scan_'
postfix = '_merlin1.h5'
scan_row = 100
scan_col = 100

data_file_list = create_file_list(data_path, prefix, postfix, sid_list)
diff_data = load_h5_data(data_file_list,roi=roi,mask=mask)

postfix = '_merlin1.txt'
mon_file_list = create_file_list(data_path, prefix, postfix, sid_list)

mon_data = load_scaler_data(mon_file_list,['sclr1_ch4'])

# First scaler reading of each scan is zero; refill with first non-zero reading
avg = np.mean(mon_data[mon_data != 0])
mon_data[mon_data==0] = avg 

sz = diff_data.shape

diff_data = (diff_data/mon_data[:,np.newaxis,np.newaxis])

diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)
'''

# get diff det params from db or enter manually 

In [17]:
energy,gamma,delta,det_dist = get_diff_det_params(int(sid_list[0]))
pix = det_params[diff_detector]

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[field] = values
/nsls2/conda/envs/2023-1.0-py310-tiled/lib/python3.10/site-packages/databroker/eventsource/shim.py:266: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

gamma, delta, dist: 14.999999999999998 20.40015714556238 500.00154456397104


In [18]:
# transform to cartesian crystal coordinates (z along hkl and x is the rocking direction)
num_angle = len(sid_list)
th_step = angles[1]-angles[0]
offset = [0,0]
data_store = 'reduced' # this will reduce the data stored. If use 'full' it can be over 100G
save_path = os.path.join(save_folder,'_rsm//')

# generate an object of the RSM class
rsm = RSM(diff_data,energy,delta,gamma,num_angle,th_step,pix,det_dist,offset)
# transform from detector coordinates to crystal coordinates
rsm.calcRSM('cryst',data_store)
# calculate strain
# 'com', center of mass, is a simple algorithm to calculate the strain. Note: There is an abitrary offset
rsm.calcSTRAIN('com')  
# show results
rsm.disp()
# save results
rsm.save(save_path)
# also save the entire object
save_file = ''.join([save_path,'rsm_.obj'])
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

Progress:   0%|          | 0/40000 [00:00<?, ?it/s]

raw det_data is deleted
qxz_data: [pos,qx,qz] with dimensions of (200, 200, 35, 758)
qyz_data: [pos,qy,qz] with dimensions of (200, 200, 100, 758)


  0%|          | 0/200 [00:00<?, ?it/s]

Directory '//data/users/current_user/HoNiO3_p1_RT_startID313841_diff_data/_rsm//' created


In [7]:
save_file = ''.join([data_path,'rsm_35_blanket_film.obj'])
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

In [34]:
plt.figure()
plt.imshow(np.log10(rsm.qxz_data[30,70,:,:].T+1e-7),clim=[-6,0])
plt.show()

In [ ]:
data_path = '//data//users//2020Q3//Ajith_2020Q3//LPE_FeS2_P4_diff//'
save_file = ''.join([data_path,'rsm.obj'])
rsm = pickle.load(open(save_file,'rb'))
rsm.disp()

In [44]:
plt.figure()
plt.imshow(np.squeeze(np.sum(np.sum(diff_data[-2,:,:,:,:],axis=0),0)))


In [32]:
# %matplotlib
interactive_map(['Pt_L','Ti_K'],stack,'rsm',np.log10(np.swapaxes(rsm.qxz_data,2,3)+1e-6),cmap='jet')

In [76]:
plt.figure()
plt.imshow(np.log10(np.swapaxes(rsm.qxz_data,2,3)[11,25,:,:]+1e-7),cmap='jet',clim=[-4.8,-2.5])

In [33]:
plt.show()

In [ ]:
# interactive figure

%matplotlib qt

# load fluorescence image stacks
# insert your own code accordingly

sum_Ge = np.sum(Ge,0)
sum_W = np.sum(W,0)
sum_Cu = np.sum(Cu,0)

cmap = 'viridis'
color = 'red'
eps = 1e-5
def onclick(event):
    global row, col
    col, row = event.xdata, event.ydata
    if col is not None and row is not None:
        row = int(np.round(row))
        col = int(np.round(col))
        ax0.clear()
        ax0.imshow(sum_Ge,cmap=cmap)
        ax0.plot(col,row,marker='o',markersize=2, color=color)
        ax0.set_title('Ge')
        ax1.clear()
        ax1.imshow(sum_W,cmap=cmap)
        ax1.plot(col,row,marker='o',markersize=2, color=color)
        ax1.set_title('W')
        ax2.clear()
        ax2.imshow(sum_Cu,cmap=cmap)
        ax2.plot(col,row,marker='o',markersize=2, color=color)
        ax2.set_title('Cu')
        ax3.clear()
        ax3.imshow(rsm.tot,cmap=cmap)
        ax3.plot(col,row,marker='o',markersize=2, color=color)
        ax3.set_title('tot')
        ax4.clear()
        ax4.imshow(np.log10(rsm.qxz_data[row,col,:,:].T+eps),cmap=cmap)
        ax4.set_title('RSM')
        fig.canvas.draw_idle()
    
    return

fig = plt.figure()
spec = fig.add_gridspec(2,3)
#fig, ax = plt.subplots(2,1)
ax0 = fig.add_subplot(spec[0,0])
ax0.imshow(clean_Ge,cmap=cmap)
ax0.set_title('Ge')

ax1 = fig.add_subplot(spec[0,1])
ax1.imshow(sum_W,cmap=cmap)
ax1.set_title('W')

ax2 = fig.add_subplot(spec[0,2])
ax2.imshow(sum_Cu,cmap=cmap)
ax2.set_title('Cu')

ax3 = fig.add_subplot(spec[1,0])
ax3.imshow(rsm.tot,cmap=cmap)
ax3.set_title('tot')

ax4 = fig.add_subplot(spec[1,1:3])
ax4.imshow(np.log10(rsm.qxz_data[0,0,:,:].T+eps),cmap=cmap)
ax4.set_title('RSM')
plt.tight_layout()

# Call click func
cid = fig.canvas.mpl_connect('button_press_event', onclick)

#plt.show()

In [ ]:
# create movie
import matplotlib
import matplotlib.animation as manimation 
matplotlib.use('AGG') 
def update_fig(row,col):

    #col = col + i
    ax0.clear()
    ax0.imshow(sum_Ge,cmap=cmap)
    ax0.plot(col,row,marker='o',markersize=2, color=color)
    ax0.set_title('Ge')
    ax1.clear()
    ax1.imshow(sum_W,cmap=cmap)
    ax1.plot(col,row,marker='o',markersize=2, color=color)
    ax1.set_title('W')
    ax2.clear()
    ax2.imshow(sum_Cu,cmap=cmap)
    ax2.plot(col,row,marker='o',markersize=2, color=color)
    ax2.set_title('Cu')
    ax3.clear()
    ax3.imshow(rsm.tot,cmap=cmap)
    ax3.plot(col,row,marker='o',markersize=2, color=color)
    ax3.set_title('tot')
    ax4.clear()
    ax4.imshow(np.log10(rsm.qxz_data[row,col,:,:].T+eps),cmap=cmap)
    ax4.set_title('RSM')
    fig.canvas.draw_idle()
    return  
     
def init():
    fig = plt.figure()
    spec = fig.add_gridspec(2,3)
    
    ax0 = fig.add_subplot(spec[0,0])
    ax0.imshow(clean_Ge,cmap=cmap)
    ax0.set_title('Ge')

    ax1 = fig.add_subplot(spec[0,1])
    ax1.imshow(sum_W,cmap=cmap)
    ax1.set_title('W')

    ax2 = fig.add_subplot(spec[0,2])
    ax2.imshow(sum_Cu,cmap=cmap)
    ax2.set_title('Cu')

    ax3 = fig.add_subplot(spec[1,0])
    ax3.imshow(rsm.tot,cmap=cmap)
    ax3.set_title('tot')

    ax4 = fig.add_subplot(spec[1,1:3])
    ax4.imshow(np.log10(rsm.qxz_data[0,0,:,:].T+eps),cmap=cmap)
    ax4.set_title('RSM')
    plt.tight_layout()
    return  

FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie', artist='hyan',
                comment='RSM variation across GAAFET')
writer = FFMpegWriter(fps=5, metadata=metadata)

init()

with writer.saving(fig, 'myfile.mp4', dpi=100):
    writer.grab_frame()
    row = 21
    col = 5
    for j in range(44):
        update_fig(row,col+j)
        writer.grab_frame()
    row = 2
    col = 28
    for j in range(40):
        update_fig(row+j,col)
        writer.grab_frame()
    #writer.finish()
 